# CommLab Final Project: AQS Implementation

## Import packages

In [92]:
from qiskit import *
import math
import random
import numpy as np
from qiskit.tools.visualization import circuit_drawer

In [93]:
qasm_sim = Aer.get_backend('qasm_simulator')
def get_measurements(qc, num_shots): 
    job = execute(qc, qasm_sim, memory = True, shots = num_shots) 
    result = job.result()
    return result

## Initializing phase

In [94]:
def bin_to_int_key(bin_key):
    # transform binary key shared by alice and trent into KAT(integer based key)
    n = len(bin_key)
    int_key = []
    zero = []
    one = []
    for i in range(0,n):
        if not bin_key[i]:
            zero.append(i)
        else:
            one.append(i)
    int_key = zero + one
    return int_key

def generate_message(n):
    message = QuantumRegister(n, name="p")
    message_measure = ClassicalRegister(n, name="p_c")
    qc = QuantumCircuit(message, message_measure)
    for i in range(n):
        random_state = quantum_info.random_statevector(dims=2, seed=None)
        qc.initialize(random_state, message[i])
    return qc

# will be replaced with QKD
def get_shared_key(n):
    r_AT = np.random.randint(0, 2, n)
    r_BT = np.random.randint(0, 2, 2*n + 1)

    K_AT = bin_to_int_key(r_AT)
    K_BT = bin_to_int_key(r_BT)
    
    return K_AT, K_BT, r_BT

def create_bell_states(n):
    A_pos = QuantumRegister(n)
    B_pos = QuantumRegister(n)  
    qc = QuantumCircuit(A_pos, B_pos)
    
    for i in range(n):
        qc.h(A_pos[i])
        qc.cnot(A_pos[i], B_pos[i])
    return qc.to_instruction(label="bell_state")

In [95]:
n = 4

P = QuantumRegister(n, name="p_original")
P_enc = QuantumRegister(n, name="p_enc")
S_A = QuantumRegister(n, name="s_a")
M_A_1 = QuantumRegister(n, name="m_a_1")
M_A_2 = QuantumRegister(n, name="m_a_2")
A_bell = QuantumRegister(n, name="alice_bell")
B_bell = QuantumRegister(n, name="bob_bell")
for_M_A = QuantumRegister(n, name="m_a_mea")
verify = QuantumRegister(1, name="verify")
verify_measure = ClassicalRegister(1, name="verify_measure")
cswap_measure = ClassicalRegister(n, name="cswap_measure")
cswap = QuantumRegister(n, name="cswap")
cswap_measure_2 = ClassicalRegister(n, name="cswap_measure_2")
cswap_2 = QuantumRegister(n, name="cswap_2")
cswap_measure_3 = ClassicalRegister(n, name="cswap_measure_3")
cswap_3 = QuantumRegister(n, name="cswap_3")
cswap_measure_4 = ClassicalRegister(n, name="cswap_measure_4")
cswap_4 = QuantumRegister(n, name="cswap_4")

qc = QuantumCircuit(P, P_enc, S_A, M_A_1, M_A_2, 
                    A_bell, B_bell, for_M_A, verify, verify_measure,
                    cswap, cswap_2, cswap_3, cswap_4, 
                    cswap_measure, cswap_measure_2, cswap_measure_3, cswap_measure_4)

get_M_A = []
for m in range(2*n):
    get_M_A.append(ClassicalRegister(1, 'mac_' + str(m)))
    qc.add_register(get_M_A[m])

measure_M_A_1 = []
measure_M_A_2 = []
for m in range(n):
    measure_M_A_1.append(ClassicalRegister(1, 'm_a_1_measure'+str(m)))
    measure_M_A_2.append(ClassicalRegister(1, 'm_a_2_measure'+str(m)))
    qc.add_register(measure_M_A_1[m])
    qc.add_register(measure_M_A_2[m])

# Step 1 : 3 copies of the message
for i in range(n):
    random_state = quantum_info.random_statevector(dims=2, seed=None)
    qc.initialize(random_state, P_enc[i])
    qc.initialize(random_state, S_A[i])
    qc.initialize(random_state, for_M_A[i])
    qc.initialize(random_state, P[i])

# keys generation
K_AT, K_BT, r_BT = get_shared_key(n)

# Step 2 : generate bell states between Alice, Bob
AB_bell = []
for i in range(n):
    AB_bell.append(A_bell[i])
for i in range(n):
    AB_bell.append(B_bell[i])
qc.append(create_bell_states(n), AB_bell)

## Signing phase

In [96]:
def chained_cnot_enc(n, K):
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)
    for i in range(n):
        if i != K[i]:
            qc.cnot(i, K[i])
    return qc.to_instruction(label="chained_cnot_enc")

def chained_cnot_dec(n, K):
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)
    print()
    for i in reversed(range(0,n)):
        if i != K[i]:
            qc.cnot(i, K[i])
    return qc.to_instruction(label="chained_cnot_dec")

def bell_measurement(n):
    q1 = QuantumRegister(n)
    q2 = QuantumRegister(n)
    qc = QuantumCircuit(q1, q2)
    for i in range(n):
        qc.cnot(q1[i], q2[i])
        qc.h(q1[i])
    return qc.to_instruction(label="bell_measurement")

In [97]:
# Step 1 : 1st copy of P --> P_enc
r_R = np.random.randint(0, 2, n)
K_R = bin_to_int_key(r_R)

qc.append(chained_cnot_enc(n, K_R), P_enc)

# Step 2 : 2nd copy of P --> P_enc --> S_A
qc.append(chained_cnot_enc(n, K_R), S_A)
qc.append(chained_cnot_enc(n, K_AT), S_A)

# Step 3 : generate one type of bell state (to transform into other bell states later)
bell_m = []

for i in range(n):
    bell_m.append(M_A_1[i])
for i in range(n):
    bell_m.append(M_A_2[i])
qc.append(create_bell_states(n), bell_m) 

# 3rd copy of P -> P_enc
qc.append(chained_cnot_enc(n, K_R), for_M_A) 

bell_m = []
# bell_m is P_enc and A_bell concatenated

for i in range(n):
    bell_m.append(A_bell[i]) # A's bell state
for i in range(n): 
    bell_m.append(for_M_A[i]) # P_enc

qc.append(bell_measurement(n), bell_m) # do bell measurement on A's bell and P_enc

In [98]:
for i in range(n):
    qc.measure(A_bell[i], get_M_A[2*i])
    qc.measure(for_M_A[i], get_M_A[2*i+1])
    qc.z(M_A_1[i]).c_if(get_M_A[2*i], 1)
    qc.x(M_A_2[i]).c_if(get_M_A[2*i+1], 1)

## Verification Phase

In [99]:
def cswap_comparison(n):
    # q1, q2 : qubits to compare
    # for_measure : the qubits to measure
    q1 = QuantumRegister(n)
    q2 = QuantumRegister(n)
    for_measure = QuantumRegister(n)
    qc = QuantumCircuit(q1, q2, for_measure)
    for i in range(n):
        qc.h(for_measure[i])
        qc.cswap(for_measure[i], q1[i], q2[i])
        qc.h(for_measure[i])
    return qc.to_instruction(label="swap test")
    # cswap measurement = 0 --> same

In [100]:
# step 1 : (P_enc, S_A) --> (P_enc, S_A)'
P_enc_S_A = [*P_enc, *S_A]
for i in range(2*n):
    if r_BT[i] == 1 :
        qc.x(P_enc_S_A[i])

# step 2 : YB = encode K_BT(P_enc, S_A)'
K_BT_2n = K_BT[:]
K_BT_2n.remove(2*n)
qc.append(chained_cnot_enc(2*n, K_BT_2n), [*P_enc, *S_A])

# decrypt : YB --> (P_enc, S_A)'
qc.append(chained_cnot_dec(2*n, K_BT_2n), [*P_enc, *S_A])

# decrypt : (P_enc, S_A)' --> (P_enc, S_A)
for i in range(2*n):
    if r_BT[i] == 1 :
        qc.x(P_enc_S_A[i])

# decode everything for swap test comparison
qc.append(chained_cnot_dec(n, K_AT), S_A)
qc.append(chained_cnot_dec(n, K_R), S_A)
qc.append(chained_cnot_dec(n, K_R), P_enc)

In [101]:
# swap test
qc.append(cswap_comparison(n), [*P_enc, *S_A, *cswap])
qc.measure(cswap, cswap_measure)

result = get_measurements(qc, 10)
counts = result.get_counts()
print(counts)

# re-encode everything : S_A, P_enc 
qc.append(chained_cnot_enc(n, K_R), S_A) 
qc.append(chained_cnot_enc(n, K_AT), S_A)

qc.append(chained_cnot_enc(n, K_R), P_enc)

{'0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0000 0000 0000 0000 0': 1, '0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0000 0000 0000 0000 0': 1, '0 0 0 0 0 0 0 0 1 0 0 1 1 0 1 0 0000 0000 0000 0000 0': 1, '0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0000 0000 0000 0000 0': 1, '0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 1 0000 0000 0000 0000 0': 1, '0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 0 0000 0000 0000 0000 0': 1, '0 0 0 0 0 0 0 0 1 1 1 0 1 0 0 0 0000 0000 0000 0000 0': 1, '0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0000 0000 0000 0000 0': 1, '0 0 0 0 0 0 0 0 1 1 1 1 0 0 1 0 0000 0000 0000 0000 0': 1, '0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0000 0000 0000 0000 0': 1}


In [102]:
# check if verifiable 
identical = True
for key in counts:
    if key[-6:-2] != '0000':
        identical = False
print(identical)

True


In [103]:
# Step 3 : Trent sends all to Bob
if identical :
    qc.x(verify)
qc.append(chained_cnot_enc(2*n+1, K_BT), [*P_enc, *S_A, *verify])

In [104]:
# Step 4 : Bob decrypts YTB and measures verify bit
qc.append(chained_cnot_dec(2*n+1, K_BT), [*P_enc, *S_A, *verify])
qc.measure(verify, verify_measure)

result = get_measurements(qc, 10)
counts = result.get_counts()
print(counts)


{'0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0000 0000 0000 0000 1': 1, '0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 1 0000 0000 0000 0000 1': 1, '0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0000 0000 0000 0000 1': 1, '0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 1 0000 0000 0000 0000 1': 1, '0 0 0 0 0 0 0 0 0 1 1 0 1 0 1 0 0000 0000 0000 0000 1': 1, '0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0 0000 0000 0000 0000 1': 1, '0 0 0 0 0 0 0 0 1 0 0 1 0 1 1 1 0000 0000 0000 0000 1': 1, '0 0 0 0 0 0 0 0 1 0 0 0 1 1 1 0 0000 0000 0000 0000 1': 1, '0 0 0 0 0 0 0 0 1 1 1 0 0 1 1 1 0000 0000 0000 0000 1': 1, '0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 0000 0000 0000 0000 1': 1}


In [105]:
verified = True
for key in counts : 
    if key[-1] != '1':
        verified = False
print(verified)

True


In [106]:
# Step 5 : Bob recovers P_enc from MA if verified is true
qc.append(bell_measurement(n), [*M_A_1, *M_A_2])
for i in range(n):
    qc.measure(M_A_1[i], measure_M_A_1[i])
    qc.measure(M_A_2[i], measure_M_A_2[i])
    qc.z(B_bell[i]).c_if(measure_M_A_1[i], 1)
    qc.x(B_bell[i]).c_if(measure_M_A_2[i], 1)
    

In [107]:
qc.draw(fold=-1)

┌──────────────────────────────────────────────┐                                                                                                                                                                                                                                                                                                                                                                                                                  
     p_original_0: ──┤ Initialize(0.21037-0.4842j,0.29757+0.79545j) ├──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                   ┌─┴──────────────────────────────────────────────┴┐                                                                                                                                                                                                                                                                                                                                                                                                                 
     p_original_1: ┤ Initialize(-0.2923-0.089584j,-0.35791-0.88229j) ├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                   ├─────────────────────────────────────────────────┤                                                                                                                                                                                                                                                                                                                                                                                                                 
     p_original_2: ┤ Initialize(-0.95895-0.10831j,-0.18162+0.18896j) ├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                   ├─────────────────────────────────────────────────┤                                                                                                                                                                                                                                                                                                                                                                                                                 
     p_original_3: ┤ Initialize(-0.058567-0.54835j,-0.16208+0.8183j) ├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                   └─┬──────────────────────────────────────────────┬┘┌───────────────────┐                          ┌───────────────────┐┌───────────────────┐     ┌───────────────────┐                                                            

In [108]:
qc.append(chained_cnot_dec(n, K_R), B_bell)
qc.append(chained_cnot_dec(n, K_R), P_enc)

qc.append(cswap_comparison(n), [*B_bell, *P_enc, *cswap_2])

qc.measure(cswap_2, cswap_measure_2)
result = get_measurements(qc, 10)
counts = result.get_counts()
print(counts)



{'1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 0000 0000 0000 0000 1': 1, '1 1 0 1 0 1 0 1 1 1 0 1 0 1 0 1 0000 0000 0000 0000 1': 1, '1 0 1 1 0 1 0 1 1 0 1 1 0 1 0 1 0000 0000 0000 0000 1': 2, '0 1 0 0 1 1 1 0 0 1 0 0 1 1 1 0 0000 0000 0000 0000 1': 1, '1 0 1 1 0 1 1 0 1 0 1 1 0 1 1 0 0000 0000 0000 0000 1': 1, '0 0 0 1 0 1 0 0 0 0 0 1 0 1 0 0 0000 0000 0000 0000 1': 1, '0 1 0 0 1 1 1 1 0 1 0 0 1 1 1 1 0000 0000 0000 0000 1': 1, '0 1 1 0 0 0 0 1 0 1 1 0 0 0 0 1 0000 0000 0000 0000 1': 1, '1 1 0 0 0 1 1 0 1 1 0 0 0 1 1 0 0000 0000 0000 0000 1': 1}


In [120]:
identical = True
for key in counts:
    if key[-11:-7] != "0000":
        identical = False
print(identical)

True


In [121]:
# re-encode everything after swap-test
qc.append(chained_cnot_enc(n, K_R), B_bell)
qc.append(chained_cnot_enc(n, K_R), P_enc)

In [122]:
if not identical: 
    print("Abort")
else : 
    print("Alice publishes KR")
    qc.append(chained_cnot_dec(n, K_R), P_enc) # Bob obtains P_original

Alice publishes KR

{'1 1 1 1 1 1 0 0 1 1 1 1 1 1 0 0 0000 0000 0000 0000 1': 1, '0 1 1 0 1 0 0 0 0 1 1 0 1 0 0 0 0000 0000 0000 0000 1': 1, '0 1 1 0 0 1 1 1 0 1 1 0 0 1 1 1 0000 0000 0000 0000 1': 1, '0 0 1 1 0 1 1 0 0 0 1 1 0 1 1 0 0000 0000 0000 0000 1': 1, '0 0 1 0 1 1 0 1 0 0 1 0 1 1 0 1 0000 0000 0000 0000 1': 1, '1 0 1 0 1 0 0 0 1 0 1 0 1 0 0 0 0000 0000 0000 0000 1': 1, '1 1 0 1 1 1 1 0 1 1 0 1 1 1 1 0 0000 0000 0000 0000 1': 1, '1 0 1 0 1 1 0 1 1 0 1 0 1 1 0 1 0000 0000 0000 0000 1': 1, '1 0 0 1 1 0 1 0 1 0 0 1 1 0 1 0 0000 0000 0000 0000 1': 2}


In [125]:
identical = True
for key in counts:
    if key[-16:-12] != "0000":
        identical = False
print(identical)

True


In [126]:
# DISPUTE RESOLVE 
# BOB : P, K_R, S_A

# TRENT decodes S_T to check if same as BOB's original key
qc.append(chained_cnot_dec(n, K_AT), S_A)
qc.append(chained_cnot_dec(n, K_R), S_A)
qc.append(cswap_comparison(n), [*P_enc, *S_A, *cswap_4])

qc.measure(cswap_4, cswap_measure_4)

result = get_measurements(qc, 10)
counts_s = result.get_counts()
print(counts_s)



{'0 1 1 1 1 1 0 1 0 1 1 1 1 1 0 1 0000 0000 0000 0000 1': 1, '1 1 1 1 1 0 0 0 1 1 1 1 1 0 0 0 0000 0000 0000 0000 1': 1, '0 0 1 1 0 1 0 1 0 0 1 1 0 1 0 1 0000 0000 0000 0000 1': 1, '1 1 0 1 1 0 1 0 1 1 0 1 1 0 1 0 0000 0000 0000 0000 1': 1, '1 1 1 1 0 0 0 1 1 1 1 1 0 0 0 1 0000 0000 0000 0000 1': 1, '1 0 0 0 1 1 1 0 1 0 0 0 1 1 1 0 0000 0000 0000 0000 1': 1, '0 1 0 0 0 1 0 1 0 1 0 0 0 1 0 1 0000 0000 0000 0000 1': 1, '0 0 0 1 0 1 1 0 0 0 0 1 0 1 1 0 0000 0000 0000 0000 1': 1, '0 0 1 1 1 0 1 0 0 0 1 1 1 0 1 0 0000 0000 0000 0000 1': 1, '1 1 0 0 0 0 1 0 1 1 0 0 0 0 1 0 0000 0000 0000 0000 1': 1}


In [127]:
identical = True
for key in counts:
    if key[-21:-17] != "0000":
        identical = False
print(identical)

True
